# Tapis v3 Gateways 2021 Hands-on

In this notebook, you will use Tapis v3 to create two systems and one application that will be used to run
an image classification job on both a VM and an HPC cluster.

To execute each `In[#]` cell, you can click inside the cell and press `Shift + Enter`

## Authenticate and initialize Tapis v3 client

First, use `tapipy` to authenticate in the **tacc** tenant and initialize the Tapis v3 client.
Enter the username and password that you were assigned for the tutorial.

In [ ]:
import datetime
import getpass
import os

username = input('Username: ')
password = getpass.getpass(prompt='Password: ', stream=None)
base_url = 'https://tacc.tapis.io'

from tapipy.tapis import Tapis
#Create python Tapis client for user
client = Tapis(base_url= base_url, username=username, password=password)
# *** Tapis v3: Call to Tokens API
client.get_tokens()
# Print Tapis v3 token
client.access_token

## Systems

In this section we create two Tapis systems, one for a training VM host and one for the stampede2 HPC cluster.

Note that although it is possible, we have not provided any login credentials in the system definitions.
Well-crafted system definitions are likely to be copied and re-used, so, for security reasons, it is recommended that
login credentials be registered using a separate API call as discussed below.

### Create a system for the VM host

In [ ]:
user_id = username
system_id = "gateways21-vm-" + user_id
vm_password = password

# Create the system definition
exec_system1 = {
  "id": system_id,
  "description": "System for testing jobs on a VM for gateways2021 tutorial",
  "systemType": "LINUX",
  "host": "129.114.19.192",
  "defaultAuthnMethod": "PASSWORD",
  "rootDir": "/home/"+user_id,
  "canExec": True,
  "jobRuntimes": [ { "runtimeType": "SINGULARITY" } ],
  "jobWorkingDir": "workdir",
  "batchScheduler": "SLURM",
  "jobIsBatch": True,
  "batchLogicalQueues": [
    {
      "name": "tapisNormal",
      "hpcQueueName": "debug",
      "maxJobs": 50,
      "maxJobsPerUser": 10,
      "minNodeCount": 1,
      "maxNodeCount": 16,
      "minCoresPerNode": 1,
      "maxCoresPerNode": 68,
      "minMemoryMB": 1,
      "maxMemoryMB": 16384,
      "minMinutes": 1,
      "maxMinutes": 60
    }
  ],
  "batchDefaultLogicalQueue": "tapisNormal"
}

# Use the client to create the system in Tapis
client.systems.createSystem(**exec_system1)

client.systems.getSystems()

client.systems.getSystem(systemId=system_id)

### Register Credentials for the VM system

After creating the system, you will need to register credentials for your username. These will be used by Tapis to
access the host. Various authentication methods can be used to access a system, such as PASSWORD and PKI_KEYS. Here we
will cover registering a password.

Then use the client to list files on the system. This will confirm that the credentials are valid.

In [ ]:
client.systems.createUserCredential(systemId=system_id, userName=user_id, password=vm_password)

client.files.listFiles(systemId=system_id, path="/")

## Applications

In order to run a job on a system you will need to create or have access to a Tapis application.